In [1]:
from add_ckpt_path import add_path_to_dust3r
add_path_to_dust3r("src/somefile.blah")
from argparse import Namespace
import numpy as np
from torchvision import transforms
from viser_utils import PointCloudViewer

args = Namespace(resolution=(512, 384), device="cuda", vis_threshold=1.5, size=512)

In [2]:
from eval.mv_recon.data import NRGBD

dataset = NRGBD(
            split="test",
            ROOT="./data/neural_rgbd",
            resolution=args.resolution,
            num_seq=1,
            full_video=True,
            kf_every=100,
        )

undo_imgnorm = transforms.Normalize((-1, -1, -1), (2, 2, 2))
scene_id = 1
sequence = dataset[scene_id]
print(f"Visualizing {dataset.scene_list[scene_id]} (scene {scene_id})")

Found 9 sequences in split test
Visualizing complete_kitchen (scene 1)


In [3]:
pts3ds_to_vis = [seq_view["pts3d"] * [-1, -1, 1] for seq_view in sequence]
colors_pytorch = [undo_imgnorm(seq_view["img"]) for seq_view in sequence]
colors_to_vis = [0.5 * (pt_img.permute(1, 2, 0).unsqueeze(0) + 1.0).numpy() for pt_img in colors_pytorch]
confidence = [np.ones(colors_to_vis[0].shape[1:3])[np.newaxis, ...] * 2 for _ in range(len(sequence))]

R_c2w = np.stack([(seq_view["camera_pose"][:3, :3].T * [-1, -1, 1]).T for seq_view in sequence])
t_c2w = np.stack([seq_view["camera_pose"][:3, 3] * [-1, -1, 1] for seq_view in sequence])

focal = [seq_view["camera_intrinsics"][0, 0] for seq_view in sequence]
pp = np.array(colors_to_vis[0].shape[1:3])[np.newaxis, ...].repeat(len(sequence), axis=0)
cam_dict = {"focal": focal, "pp": pp, "R": R_c2w, "t": t_c2w}

edge_colors = [None] * len(pts3ds_to_vis)

In [4]:
viewer = PointCloudViewer(
    None,
    None,
    pts3ds_to_vis,
    colors_to_vis,
    confidence,
    cam_dict,
    device=args.device,
    edge_color_list=edge_colors,
    show_camera=True,
    vis_threshold=args.vis_threshold,
    size = args.size,
    only_show_edge=False
)
viewer.run()

╭──────────────── viser ────────────────╮
│             ╷                         │
│   HTTP      │ http://localhost:8080   │
│   Websocket │ ws://localhost:8080     │
│             ╵                         │
╰───────────────────────────────────────╯

KeyboardInterrupt: 